In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings 
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0) 

In [3]:
llm.invoke("Hi, how are you").content

"Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"

## Predifined Tools

In [4]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper = WikipediaAPIWrapper()
tool = WikipediaQueryRun(api_wrapper=api_wrapper)
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Vector database
Summary: A vector database, vector store or vector search engine is a database that stores and retrieves embeddings of data in vector space. Vector databases typically implement approximate nearest neighbor algorithms so users can search for records semantically similar to a given input, unlike traditional databases which primarily look up records by exact match. Use-cases for vector databases include similarity search, semantic search, multi-modal search, recommendations engines, object detection, and retrieval-augmented generation (RAG).
Vector embeddings are mathematical representations of data in a high-dimensional s

## Create a custom tool

In [5]:
from langchain_core.tools import tool

@tool
def get_word_len(word: str) -> int:
    """Returns length of the word."""
    return len(word)

In [6]:
get_word_len.invoke("Ankit Anand")

11

In [7]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b
    

In [8]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [9]:
multiply.invoke({"a": 10, "b": 20})

200

## Concept of Agents

## This agent class is from first version of the langchain

In [10]:
from langchain_tavily import TavilySearch
search = TavilySearch()
# search.invoke("What is the wether in NY")

/Users/ankitanand/Documents/Learnings/LangGraph/Langgraph-end-to-end/Langgraph_Basics_02/.venv/lib/python3.11/site-packages/langchain_tavily/tavily_research.py:97: UserWarning: Field name "output_schema" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]
/Users/ankitanand/Documents/Learnings/LangGraph/Langgraph-end-to-end/Langgraph_Basics_02/.venv/lib/python3.11/site-packages/langchain_tavily/tavily_research.py:97: UserWarning: Field name "stream" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]


In [11]:
res = search.invoke("What is the wether in NY")

In [12]:
type(res)

dict

In [13]:
res

{'query': 'What is the wether in NY',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.theweathernetwork.com/en/city/us/new-york/new-york/current',
   'title': 'New York, NY Current Weather',
   'content': "# New York, NY Current Weather. * January outlook: Winter hits pause in Canada before striking back. * When is the coldest stretch of the year in your corner of Canada? Canada's National Forecast - January 4, 2026. Calgary urged to conserve water after major pipe failure. * January can churn out the year's biggest snows across Canada. * Start your January with a supermoon and the best winter meteor shower. * Why you should embrace the great outdoors as your New Year's resolution. * The Winter Classic is a fun NHL tradition, even through these weather events. Calgary residents share rescue experiences after water main break. * Wildfires, heat, ice and storms: Canada feels weather wrath in 2025. Canada's sunniest cities may surprise you. 

In [14]:
res['results'][0]['content']

"# New York, NY Current Weather. * January outlook: Winter hits pause in Canada before striking back. * When is the coldest stretch of the year in your corner of Canada? Canada's National Forecast - January 4, 2026. Calgary urged to conserve water after major pipe failure. * January can churn out the year's biggest snows across Canada. * Start your January with a supermoon and the best winter meteor shower. * Why you should embrace the great outdoors as your New Year's resolution. * The Winter Classic is a fun NHL tradition, even through these weather events. Calgary residents share rescue experiences after water main break. * Wildfires, heat, ice and storms: Canada feels weather wrath in 2025. Canada's sunniest cities may surprise you. * Snowmobiler dies in avalanche in the Rocky Mountains near Tumbler Ridge, B.C. Looking back on Canada's top weather events of 2025. * Coldest New Year's Day in over a decade expected in parts of Canada."

In [15]:
tools = [search]

In [16]:
from langsmith import Client; client = Client(); prompt = client.pull_prompt("hwchase17/openai-functions-agent")

In [17]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [18]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [19]:
# Modern LangChain approach using LangGraph-based agents
from langchain.agents import create_agent

# create_agent now returns a CompiledStateGraph (LangGraph-based)
agent = create_agent(llm, tools, system_prompt="You are a helpful assistant")

In [20]:
agent.invoke({"messages": [{"role": "user", "content": "What is the Capital of Jharkhand"}]})

{'messages': [HumanMessage(content='What is the Capital of Jharkhand', additional_kwargs={}, response_metadata={}, id='934c6a1d-51e4-4d67-82ac-c84b0a69c0ba'),
  AIMessage(content='The capital of Jharkhand is Ranchi.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 1201, 'total_tokens': 1212, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CuOlEX5mcIpab23JXiNePjREiVHBE', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b8aa4-56fd-7603-8cb3-66f577aebf6a-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 1201, 'output_tokens': 11, 'total_tokens': 1212, 'input_token_details': {'audio': 0, 

In [21]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS 
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [22]:
loader = WebBaseLoader("https://docs.langchain.com/langsmith/agent-server#agent-server")
docs = loader.load()

In [23]:
docs

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/agent-server#agent-server', 'title': 'Agent Server - Docs by LangChain', 'language': 'en'}, page_content='Agent Server - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationDeployment componentsAgent ServerGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupReferenceOverviewTest locallyApp developmentCloud quickstartConfigure app for deploymentApplication structureSetupDeployment componentsOverviewAgent ServerData planeControl planeRebuild graph at runtimeInteract with a deployment using RemoteGraphAdd semantic search to your agent deploymentAdd TTLs to your applicationConfigure Agent Server for scaleImplement a CI/CD pipelineDeployment guidesCloudWith control planeStandalone serversApp developmentData modelsCore capabilitiesTutorialsStudioOverviewQuickstartRuns, assistants, threadsTraces, datas

In [24]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
).split_documents(docs)

In [25]:
vector = FAISS.from_documents(documents, embeddings)

In [26]:
retriever = vector.as_retriever()

In [27]:
retriever.invoke("What is langgraph studio")[0]

Document(id='402e92ec-721d-4749-9902-4194351cd06f', metadata={'source': 'https://docs.langchain.com/langsmith/agent-server#agent-server', 'title': 'Agent Server - Docs by LangChain', 'language': 'en'}, page_content='\u200bApplication structure\nTo deploy an Agent Server application, you need to specify the graph(s) you want to deploy, as well as any relevant configuration settings, such as dependencies and environment variables.\nRead the application structure guide to learn how to structure your LangGraph application for deployment.\n\u200bParts of a deployment\nWhen you deploy Agent Server, you are deploying one or more graphs, a database for persistence, and a task queue.\n\u200bGraphs\nWhen you deploy a graph with Agent Server, you are deploying a “blueprint” for an Assistant.\nAn Assistant is a graph paired with specific configuration settings. You can create multiple assistants per graph, each with unique settings to accommodate different use cases\nthat can be served by the same

In [28]:
from langchain_core.tools import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langgraph_studio",
    "Search information about langgraph. For any questions about Langgraph, you must use this tool"
)

In [29]:
tools = [search, retriever_tool]

In [30]:
# Modern LangChain approach using LangGraph-based agents
from langchain.agents import create_agent

# create_agent now returns a CompiledStateGraph (LangGraph-based)
agent = create_agent(llm, tools, system_prompt="You are a helpful assistant")

In [31]:
agent.invoke({"messages": [{"role": "user", "content": "Hi! what is langgraph studio and how to use it?"}]})

{'messages': [HumanMessage(content='Hi! what is langgraph studio and how to use it?', additional_kwargs={}, response_metadata={}, id='7b9e980e-9872-4de2-80ee-c6ec489e90f5'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 1250, 'total_tokens': 1276, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CuOlK1CHuPaCftiOKtKqsE4doj8l6', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b8aa4-6b85-74b2-9f63-0f159bbb7702-0', tool_calls=[{'name': 'langgraph_studio', 'args': {'query': 'What is Langgraph Studio and how to use it?'}, 'id': 'call_49BMapNIAzjZAZZNe6LLRQUt', 'type': 'tool_call'}], 

In [32]:
agent.invoke({"messages": [{"role": "user", "content": "what is the weather temperature in Frederick, MD in celsius"}]})

{'messages': [HumanMessage(content='what is the weather temperature in Frederick, MD in celsius', additional_kwargs={}, response_metadata={}, id='4e5220ca-dcef-49c3-824e-438034b2aed1'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 1249, 'total_tokens': 1275, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CuOlTUhE5Dd8JTzwtBJKEjmoSZreJ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b8aa4-8f7d-7e30-a773-52bcb48d8dc9-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'current weather temperature in Frederick, MD', 'topic': 'general'}, 'id': 'call_AWp9WpL4YFxLiJDdMy5i

In [33]:
# Updated imports for modern LangChain
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.tools import Tool
from langchain_core.prompts import PromptTemplate
from langchain.agents import create_agent  # create_react_agent is deprecated, use create_agent

In [34]:
google_search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="web_search",  # Fixed: removed space, tool names must match pattern ^[a-zA-Z0-9_-]+$
        func=google_search.run,
        description="useful for when you need to ask with search",
        verbose=True
    )
]

In [35]:
# Removed unnecessary imports that were causing errors
# The nt module is Windows-specific and not needed here

template = f"""Answer the following question as best as you can. You have access to the following tools: {tools} 
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [tool_names]
Action_input: the input to the Action
Observation: the result of the Action
...(this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question 
Begin!
Question: {{input}}
Thought: {{agent_scratchpad}}"""

In [36]:
prompt = PromptTemplate.from_template(template)

In [37]:
# Modern create_agent doesn't accept custom prompt templates like the old create_react_agent
# It only accepts system_prompt as a string parameter
# The custom ReAct prompt pattern is now built-in to LangGraph agents

search_agent = create_agent(
    llm, 
    tools, 
    system_prompt="Answer the following question as best as you can. Use available tools to search for information when needed."
)

In [38]:
# AgentExecutor is deprecated - create_agent already returns a ready-to-use CompiledStateGraph
# The search_agent is already executable, no need to wrap it in AgentExecutor
# You can directly invoke it with: search_agent.invoke({"messages": [{"role": "user", "content": "your query"}]})

In [39]:
search_agent.invoke({"messages": [{"role": "user", "content": "where is the hometown of the 2007 T20 Cricket World cup caption winner and his score?"}]})

content="India won the tournament, beating Pakistan in the final at Johannesburg by 5 runs. 2007 ICC World Twenty20. Tournament logo. Dates, 11 – ... Final, Johannesburg, September 24, 2007, ICC World Twenty20, IND Flag IND 157/5, PAK Flag PAK (19.3/20 ov, T:158) 152, India won by 5 runs. Indian captain MS Dhoni celebrates after winning the 2007 T20 World Cup final against Pakistan. Picture by Getty Images. By Ali Asgar Nalwala ... India won by defeating Pakistan by five runs. The teams had previously played each other in a Group-D match of the tournament, which was also won by India. On this day in 2007, Chris Gayle became the first centurion in T20I history, smashing 117 off just 57 balls against South Africa in the T20 ... Matthew Hayden of Australia scored the most runs in the tournament with 265, while Pakistan's Umar Gul took the most wickets, with a total of 7. Rohit Sharma has become the only Indian player to win two T20 World Cups, first in 2007 under MS Dhoni and then in 2024

{'messages': [HumanMessage(content='where is the hometown of the 2007 T20 Cricket World cup caption winner and his score?', additional_kwargs={}, response_metadata={}, id='d50b5226-23b3-4ea6-94c6-54a3b9bb533a'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 94, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CuOlWq3LUG18Xby4VCOrQMxlo0YiD', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b8aa4-9e27-7600-a855-53c87d89b2cf-0', tool_calls=[{'name': 'web_search', 'args': {'__arg1': '2007 T20 Cricket World Cup captain winner and his score'}, 'id': 'call_1ebsrRlj

### Self-Ask with Search tool

In [ ]:
tools = [
    Tool(
        name="web_search",  # Fixed: removed space, tool names must match pattern ^[a-zA-Z0-9_-]+$
        func=google_search.run,
        description="useful for when you need to ask with search",
        verbose=True
    )
]

In [41]:
from langsmith import Client
client = Client()
prompt = client.pull_prompt("hwchase17/self-ask-with-search")

In [43]:
print(prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [ ]:
# Extract the template string from the PromptTemplate object
# system_prompt parameter expects a string, not a PromptTemplate object
agent = create_agent(
    llm, 
    tools, 
    system_prompt=prompt.template  # Get the template string from the PromptTemplate
)

In [47]:
ans = agent.invoke({"messages": [{"role": "user", "content": "where is the hometown of the current prime minister of India"}]})

In [53]:
ans

{'messages': [HumanMessage(content='where is the hometown of the current prime minister of India', additional_kwargs={}, response_metadata={}, id='695a25ea-81d7-4867-9626-c6db8dc68c4a'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 1248, 'total_tokens': 1269, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CuadsfcUcc6I6jQTSydSHAz3yWJXj', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b8d5d-39b4-7050-af32-13cf8ca3df05-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'current prime minister of India hometown'}, 'id': 'call_FXcdLNN4p282SESIpgvW1dbA', 'type': 'tool_ca